# 1. Merge all files of sonhdothi.net

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import pandas as pd

import selenium
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta
import os

import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [23]:
folder_path = "D:\\OneDrive\\KiotViet\\Python_for_work\\KFinance\\CSV_crawl_data"
file_list = os.listdir(folder_path)

merged_df = pd.DataFrame()

# Loop through each file in the folder
for file in file_list:
    if "dothinet_" in file and file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)

        # Specify the encoding as 'utf-8' or the appropriate encoding for your files
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
            merged_df = pd.concat([merged_df, df], ignore_index=True)
        except UnicodeDecodeError:
            print(f"Error reading file: {file_path}. Skipping.")

print(len(merged_df))

1438


In [24]:
merged_df = merged_df.dropna()
merged_df.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_dothinet.csv", index=False, mode = 'w', encoding="utf-8-sig")

# 2. Clean data 

In [25]:
df = pd.read_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_dothinet.csv")
df.head()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 1437


In [26]:
df = df[~df['Area'].str.contains('Không xác định')]
df = df[~df['Price'].str.contains('Thỏa thuận')]
df = df.drop_duplicates()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 1236


2.0 Xử lý diện tích

In [27]:
df['Area'].head()

1     Diện tích\n:\n70 m²
2    Diện tích\n:\n113 m²
3    Diện tích\n:\n300 m²
4    Diện tích\n:\n178 m²
5    Diện tích\n:\n200 m²
Name: Area, dtype: object

In [28]:
df['area_value'] = df['Area'].apply(lambda x: float(x.replace('Diện tích\n:\n', '').replace(' m²', ''))) #.replace('.', '')

In [29]:
df.head()

,Title,Price,Area,Location,type,area_value
1,CHO THUÊ MẶT BẰNG MẶT TIỀN TẦNG LỬNG GIÁ RẺ ĐƯ...,Giá\n:\n13 Triệu/tháng,Diện tích\n:\n70 m²,Vị trí\n:\nHải Châu - Đà Nẵng,cua_hang_kiot,70.0
2,"🔴Cho thuê mặt bằng ngay trường tiểu học, bán t...",Giá\n:\n16 Triệu/tháng,Diện tích\n:\n113 m²,Vị trí\n:\nBiên Hòa - Đồng Nai,cua_hang_kiot,113.0
3,"🔴Cho thuê mặt bằng ngang 10m, ngay coopmart, đ...",Giá\n:\n60 Triệu/tháng,Diện tích\n:\n300 m²,Vị trí\n:\nBiên Hòa - Đồng Nai,cua_hang_kiot,300.0
4,"Cho thuê shophouse khu Jamona City,Đào Trí,P.P...",Giá\n:\n18 Triệu/tháng,Diện tích\n:\n178 m²,Vị trí\n:\nQuận 7 - Hồ Chí Minh,cua_hang_kiot,178.0
5,🔴Cho thuê mặt bằng ngang gần 10m đường Phạm Vă...,Giá\n:\n70 Triệu/tháng,Diện tích\n:\n200 m²,Vị trí\n:\nBiên Hòa - Đồng Nai,cua_hang_kiot,200.0


2.1 Xử lý price

In [30]:
#Tạo cột 'price_per_m2'

df['price_per_m2'] = df['Price'].apply(lambda x: str(x.replace('Giá\n:\n', '')))

def convert_price_string_to_float(price_string):
    return float(price_string.replace(',', '.'))

price_pattern = re.compile(r'([\d,]+) (Triệu/tháng)')
price_pattern2 = re.compile(r'([\d,]+) (Triệu/m²/tháng)')
price_pattern_thou = re.compile(r'([\d,]+) (Nghìn/m²/tháng)')

#1.Nếu Price ghi X triệu/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành X*1000000/Y
df['price_value_trieu'] = df['price_per_m2'].apply(lambda x: convert_price_string_to_float(price_pattern.search(x).group(1)) if price_pattern.search(x) else None)
df['price_per_m2_trieu'] = 1000000 * df['price_value_trieu'] / df['area_value']
# df['price_per_m2_trieu']

#2. Nếu Price ghi X triệu/m2/tháng thì cột 'price_per_m2' là X*1000000
df['price_value_trieu2'] = df['price_per_m2'].apply(lambda x: convert_price_string_to_float(price_pattern2.search(x).group(1)) if price_pattern2.search(x) else None)
df['price_per_m2_trieu2'] = 1000000 * df['price_value_trieu2']
# df['price_per_m2_trieu2'] 

#3.Nếu Price ghi X Nghìn/m2/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành X*1000
df['price_value_nghin'] = df['price_per_m2'].apply(lambda x: convert_price_string_to_float(price_pattern_thou.search(x).group(1)) if price_pattern_thou.search(x) else None)
df['price_per_m2_nghin'] = np.where(df['price_value_nghin'] < 10000,df['price_value_nghin'] * 1000,df['price_value_nghin']) 
# df['price_per_m2_nghin']



In [31]:
#4. Từ các cột "price_per_m2", "price_per_m2_trieu", "price_per_m2_ty" ta tìm được price_per_m2_final

def calculate_price_per_m2_final(row):
    if not pd.isna(row['price_per_m2_trieu']):
        return row['price_per_m2_trieu']
    elif not pd.isna(row['price_per_m2_trieu2']):
        return row['price_per_m2_trieu2']
    elif not pd.isna(row['price_per_m2_nghin']):
        return row['price_per_m2_nghin']
    else:
        return np.nan

# Áp dụng hàm tùy chỉnh để tạo cột mới
df['price_per_m2_final'] = df.apply(calculate_price_per_m2_final, axis=1)
df = df.drop(columns=['price_per_m2_nghin','price_per_m2_trieu2','price_per_m2_trieu','price_value_nghin','price_value_trieu2']).dropna()
df.head()

,Title,Price,Area,Location,type,area_value,price_per_m2,price_value_trieu,price_per_m2_final
1,CHO THUÊ MẶT BẰNG MẶT TIỀN TẦNG LỬNG GIÁ RẺ ĐƯ...,Giá\n:\n13 Triệu/tháng,Diện tích\n:\n70 m²,Vị trí\n:\nHải Châu - Đà Nẵng,cua_hang_kiot,70.0,13 Triệu/tháng,13.0,185714.285714
2,"🔴Cho thuê mặt bằng ngay trường tiểu học, bán t...",Giá\n:\n16 Triệu/tháng,Diện tích\n:\n113 m²,Vị trí\n:\nBiên Hòa - Đồng Nai,cua_hang_kiot,113.0,16 Triệu/tháng,16.0,141592.920354
3,"🔴Cho thuê mặt bằng ngang 10m, ngay coopmart, đ...",Giá\n:\n60 Triệu/tháng,Diện tích\n:\n300 m²,Vị trí\n:\nBiên Hòa - Đồng Nai,cua_hang_kiot,300.0,60 Triệu/tháng,60.0,200000.000000
4,"Cho thuê shophouse khu Jamona City,Đào Trí,P.P...",Giá\n:\n18 Triệu/tháng,Diện tích\n:\n178 m²,Vị trí\n:\nQuận 7 - Hồ Chí Minh,cua_hang_kiot,178.0,18 Triệu/tháng,18.0,101123.595506
5,🔴Cho thuê mặt bằng ngang gần 10m đường Phạm Vă...,Giá\n:\n70 Triệu/tháng,Diện tích\n:\n200 m²,Vị trí\n:\nBiên Hòa - Đồng Nai,cua_hang_kiot,200.0,70 Triệu/tháng,70.0,350000.000000


2.2. Xử lý địa chỉ

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 1 to 1436
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               1140 non-null   object 
 1   Price               1140 non-null   object 
 2   Area                1140 non-null   object 
 3   Location            1140 non-null   object 
 4   type                1140 non-null   object 
 5   area_value          1140 non-null   float64
 6   price_per_m2        1140 non-null   object 
 7   price_value_trieu   1140 non-null   float64
 8   price_per_m2_final  1140 non-null   float64
dtypes: float64(3), object(6)
memory usage: 89.1+ KB


In [33]:
df['address'] = df['Title'].apply(lambda x: str(x.replace('Vị trí\n:\n', '')))
df[['district', 'city']] = df['Location'].apply(lambda x: str(x.replace('Vị trí\n:\n', ''))).str.split(' - ', expand=True)
# df.head()


In [34]:
list_api_keys = ["1EotnhALO_Xf7d5mlOVefECdfpbgPG_Dik0TeOcQauQ",
"92hxCAU523jWUFRXMhM-p3w66P3PxYfY_tZm9pPeShU",
"KwvOafjkobVhF8Iz63lMyNooS-hjaYgmHdZrqVVVHWw",
"N5qb2q5dSfly3fnqWMsDPqVMWUd1yRbTBxbT34hvw_g",
"SsQNhtNfemZJr_FzCYZlub29VeTTjymT3Qm6syC50W8",
"br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
"spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
"0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
"UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
"c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE",
"WCpDyLiAACoL9zwSFQQa_HboP-HdWXSM601W1N02aeg",
"faJrm5egL12NmLivrDsbqz44I47DhbJQUIg89L3TdIo",
"cdXQHZ3zzsZv-n04kjimcml7e3BRKcR2H3wl8XPi6SU",
"RBRLSkMgpwy6QF0jKsXs-jXEliqtAwwD_jd0DbHrEN8",
"Dd-n66bTKOIKeQ0H6nRvIOMWatnJ_viPxKMNGpxsFa4",
"q-Ao_RRfZNyfQPp2XsnYGbrOItF6SlXN0jObPIKVnk",
"ufLjrWIrvjyLHzL1TZIEOZjKpgolDMBUPXUwoIblgs0",
"tvjh1g0Xygp6DMsH8n5hX41jgbNh3VE61MsnWHYkfZw",
"d_8_FhktHjeMh91gB9bBKGHgMJFC6auZqHThhlHL9Hw",
"P1RAwTjkXvZUQLjHyi06Zm1YP1KeaZMJdjaNN1ZeOeE",
"DQV3KQ2U9DLoro0jMQUoIr-PQMKBpKIEpOqXzF1A498",
"6IcCytBh4PkKOC3YE2Smw4IGdAyTD7XTtsmgtxF-kdY",
"PgxoArILmRDFQreVp3pC6bexYfcKIVwMMN4oKpa8VOo",
"mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"]

In [35]:
# Hàm gọi api here để lấy thông tin vị trí từ địa chỉ chi tiết
import requests

def get_location(address, api_key):
    #api_key = "mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"

    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách

        if response.status_code == 429:
            return {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
        else: 
            data = response.json()    
            # Xử lý kết quả và trả về
            if data == {'items': []}: 
                list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "True" }
            else: 
                result = data['items'][0]
                position = result['position']
                lat = position.get('lat', '')
                long = position.get('lng', '')
                address_info = result['address']
                city = address_info.get('county', '')
                district = address_info.get('city', '')
                ward = address_info.get('district', '')
                street = address_info.get('street', '')
                houseNumber = address_info.get('houseNumber', '')
                list_address = {"lat": lat, "lng": long, "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
           # Nếu thử xảy ra lỗi trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
        list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
           
    return list_address

# Hàm xử lí dữ liệu đầu vào là dataframe
def out_put(df, api_key):
    house_numbers = []
    streets = []
    wards = []
    districts = []
    cities = []
    progresses = []
    
    for address in df["address"]:
        address_info = get_location(address,api_key)
        house_numbers.append(address_info["House_number"])
        streets.append(address_info["Street"])
        wards.append(address_info["Ward"])
        districts.append(address_info["District"])
        cities.append(address_info["City"])
        progresses.append(address_info["progress"])

    # for index, row in df.iterrows():
    #     title_info = get_location(row["Title"], api_key)
    #     address_info = get_location(row["address"], api_key)

    #     house_numbers.append(title_info["House_number"])
    #     streets.append(title_info["Street"])
    #     wards.append(title_info["Ward"])

    #     # Lấy thông tin City và District từ cột "address"
    #     districts.append(address_info["District"])
    #     cities.append(address_info["City"])

    #     progresses.append(address_info["progress"])
            
    # for address, title in zip(df["address"], df["Title"]):
    #     title_info = get_location(title, api_key)
    #     house_numbers.append(title_info["House_number"])
    #     streets.append(title_info["Street"])
    #     wards.append(title_info["Ward"])
    #     address_info = get_location(address, api_key)
    #     districts.append(address_info["District"])
    #     cities.append(address_info["City"])
    #     progresses.append(address_info["progress"])
    
    df["House_number"] = house_numbers
    df["Street"] = streets
    df["Ward"] = wards
    df["District"] = districts
    df["City"] = cities
    df["progress"] = progresses

    return df

In [36]:
df_test = df

In [37]:
# Calculate the number of groups
num_groups = len(df_test) // 1000

# Calculate the number of remaining rows
remaining_rows = len(df_test) % 1000

# Create a list to store the new dataframes
dfs = []

# Split the dataframe into groups of two rows each
for i in range(num_groups):
    start_index = i * 1000
    end_index = start_index + 1000
    df_group = df_test[start_index:end_index].copy()
    dfs.append(df_group)

# Add the remaining rows to the last dataframe
if remaining_rows > 0:
    last_df = df_test[-remaining_rows:].copy()
    dfs.append(last_df)

In [38]:
# tiến hành chạy multi thread
import threading
import time

final = []
def out_put_with_time(df, api_key):
    start_time = time.time()
    out_put(df, api_key)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time: {execution_time/60} phút")

for df , api_key in zip(dfs, list_api_keys):
    thread = threading.Thread(target=out_put_with_time, args=(df,api_key))
    thread.start()
    thread.join()
    final.append(df)

print("Done")

Time: 3.1814946293830872 phút
Time: 0.464534854888916 phút
Done


In [39]:
merged_df = pd.concat(final, ignore_index=True)

#check case false
# merged_df[merged_df["progress"] == "False"]


In [40]:
merged_df['City'] = np.where(merged_df['City'].str.contains('[a-zA-Z]'),merged_df['City'],merged_df['city'])
merged_df['District'] = np.where(merged_df['District'].str.contains('[a-zA-Z]'),merged_df['District'],merged_df['district'])
merged_df['price_value_trieu'] = merged_df['price_per_m2_final']*merged_df['area_value']/1000000

In [41]:
df_final_dothinet = merged_df[['City','District','Ward','Street','address','area_value','price_value_trieu','price_per_m2_final']]
df_final_dothinet.head()

,City,District,Ward,Street,address,area_value,price_value_trieu,price_per_m2_final
0,Đà Nẵng,Quận Hải Châu,Phường Bình Hiên,Đường 2 Tháng 9,CHO THUÊ MẶT BẰNG MẶT TIỀN TẦNG LỬNG GIÁ RẺ ĐƯ...,70.0,13.0,185714.285714
1,Đồng Nai,Biên Hòa,,,"🔴Cho thuê mặt bằng ngay trường tiểu học, bán t...",113.0,16.0,141592.920354
2,Đồng Nai,Biên Hòa,,,"🔴Cho thuê mặt bằng ngang 10m, ngay coopmart, đ...",300.0,60.0,200000.000000
3,Hồ Chí Minh,Quận 7,Phường Phú Thuận,Đường Đào Trí,"Cho thuê shophouse khu Jamona City,Đào Trí,P.P...",178.0,18.0,101123.595506
4,Đồng Nai,Biên Hòa,,,🔴Cho thuê mặt bằng ngang gần 10m đường Phạm Vă...,200.0,70.0,350000.000000


In [42]:
df_final_dothinet.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\final_bds_dothinet.csv", index=False, mode = 'w', encoding="utf-8-sig")